In [1]:
# python required >= 3.5 version
import sys
assert sys.version_info >=(3,5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

import numpy as np
import os
import pandas as pd
# make the output satable accros run
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

ROOT = "."
chapter_id = "DNN"
image_path = os.path.join(ROOT,"images",chapter_id)
os.makedirs(image_path,exist_ok=True)

def save_fig(fig_id,tigh_layout=True,fig_extension="png",resolution=300):
    path = os.path.join(image_path,fig_id + "." + fig_extension)
    print("save figure ",fig_id)
    if tigh_layout:
        plt.tight_layout()
    plt.savefig(path,format=fig_extension,dpi=resolution)


# Gradient Clipping

semua keras di optimizers clipnorm atau clipvalue

In [2]:
# clip value can change the orientation gradient vector
optimizer = keras.optimizers.SGD(learning_rate=1e-3,clipvalue=1.0)

In [3]:
# if not want gradientn vector change the orientation value , we can use
# clipnorm thats ill clip gradient if the l2 forn greater than theresold 
# which is pick
optimizer = keras.optimizers.SGD(learning_rate=1e-3,clipnorm=1.0)

# Reusing pretrained layers

Let's split the fashion MNIST training set in two:
* `X_train_A`: all images of all items except for sandals and shirts (classes 5 and 6).
* `X_train_B`: a much smaller training set of just the first 200 images of sandals or shirts.

The validation set and the test set are also split this way, but without restricting the number of images.

We will train a model on set A (classification task with 8 classes), and try to reuse it to tackle set B (binary classification). We hope to transfer a little bit of knowledge from task A to task B, since classes in set A (sneakers, ankle boots, coats, t-shirts, etc.) are somewhat similar to classes in set B (sandals and shirts). However, since we are using `Dense` layers, only patterns that occur at the same location can be reused (in contrast, convolutional layers will transfer much better, since learned patterns can be detected anywhere on the image, as we will see in the CNN chapter).

In [5]:
# get dataset fashion_mnist dataset
(X_train_full,y_train_full),(X_test,y_test) = keras.datasets.fashion_mnist.load_data()
# standarisasi train data
X_train_full = X_train_full/255.0

# split train data to train and validasi data
X_valid,X_train = X_train_full[:5000], X_train_full[5000:]
y_valid,y_train = y_train_full[:5000], y_train_full[5000:]

In [7]:
def split_dataset(X,y):
    y_5_or_6 = (y==5) | (y==6) # sandls or shirts
    y_A  = y[~y_5_or_6]
    y_A[y_A >6]-=2 # class indices 7,8,9, should be moved to 5,6,7
    y_B = (y[y_5_or_6] == 6).astype(np.float32)#binary classification task: is its shirt(class 6)?
    return ((X[~y_5_or_6],y_A),
           (X[y_5_or_6],y_B))

In [8]:
(X_train_A,y_train_A),(X_train_B,y_train_B) = split_dataset(X_train,y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [9]:
X_train_A.shape

(43986, 28, 28)

In [11]:
X_test_A.shape

(8000, 28, 28)

In [13]:
X_valid_A.shape

(4014, 28, 28)

In [14]:
X_train_B.shape

(200, 28, 28)

In [15]:
y_train_A[:30]

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [16]:
y_train_B[:30]

array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)

In [18]:
tf.random.set_seed(42)
np.random.seed(42)

# Normal sequential model

In [20]:
# arsitektur model A
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28,28]))
for n_hidden in (300,100,50,50,50):
    model_A.add(keras.layers.Dense(n_hidden,activation="selu"))
model_A.add(keras.layers.Dense(8,activation="softmax"))

In [21]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [24]:
history_model_A = model_A.fit(X_train_A,y_train_A,epochs=20,
                              validation_data=[X_valid_A,y_valid_A])

Epoch 1/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.3527 - accuracy: 0.8783 - val_loss: 0.3380 - val_accuracy: 0.8799
Epoch 2/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.3169 - accuracy: 0.8902 - val_loss: 0.3041 - val_accuracy: 0.8944
Epoch 3/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.2972 - accuracy: 0.8969 - val_loss: 0.2864 - val_accuracy: 0.9033
Epoch 4/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.2834 - accuracy: 0.9025 - val_loss: 0.2785 - val_accuracy: 0.9056
Epoch 5/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.2729 - accuracy: 0.9061 - val_loss: 0.2697 - val_accuracy: 0.9093
Epoch 6/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2645 - accuracy: 0.9093 - val_loss: 0.2669 - val_accuracy: 0.9088
Epoch 7/20
1375/1375 [==============================] - 2s 2ms/step - loss: 0.2572 - accuracy: 0.9115 - val_loss: 0.2673 - val_accuracy:

In [40]:
model_A.save("my_model_A.h5")

In [32]:
[i for i in dir(keras.activations)if not i.startswith("_")]

['deserialize',
 'elu',
 'exponential',
 'gelu',
 'get',
 'hard_sigmoid',
 'linear',
 'relu',
 'selu',
 'serialize',
 'sigmoid',
 'softmax',
 'softplus',
 'softsign',
 'swish',
 'tanh']

In [33]:
#arsitektur model_B
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28,28]))
for n_hidden in (300,100,50,50,50):
    model_B.add(keras.layers.Dense(n_hidden,activation="selu"))
model_B.add(keras.layers.Dense(1,activation="sigmoid"))

In [37]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [38]:
history_model_b=model_B.fit(X_train_B,y_train_B,epochs=20,
                            validation_data=[X_valid_B,y_valid_B])

Epoch 1/20
7/7 [==============================] - 1s 28ms/step - loss: 0.9573 - accuracy: 0.4650 - val_loss: 0.6314 - val_accuracy: 0.6004
Epoch 2/20
7/7 [==============================] - 0s 10ms/step - loss: 0.5692 - accuracy: 0.7450 - val_loss: 0.4784 - val_accuracy: 0.8529
Epoch 3/20
7/7 [==============================] - 0s 10ms/step - loss: 0.4503 - accuracy: 0.8650 - val_loss: 0.4102 - val_accuracy: 0.8945
Epoch 4/20
7/7 [==============================] - 0s 10ms/step - loss: 0.3879 - accuracy: 0.8950 - val_loss: 0.3647 - val_accuracy: 0.9178
Epoch 5/20
7/7 [==============================] - 0s 10ms/step - loss: 0.3435 - accuracy: 0.9250 - val_loss: 0.3300 - val_accuracy: 0.9320
Epoch 6/20
7/7 [==============================] - 0s 10ms/step - loss: 0.3081 - accuracy: 0.9300 - val_loss: 0.3019 - val_accuracy: 0.9402
Epoch 7/20
7/7 [==============================] - 0s 10ms/step - loss: 0.2800 - accuracy: 0.9350 - val_loss: 0.2804 - val_accuracy: 0.9422
Epoch 8/20
7/7 [===========

In [39]:
model_B.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_6 (Dense)             (None, 300)               235500    
                                                                 
 dense_7 (Dense)             (None, 100)               30100     
                                                                 
 dense_8 (Dense)             (None, 50)                5050      
                                                                 
 dense_9 (Dense)             (None, 50)                2550      
                                                                 
 dense_10 (Dense)            (None, 50)                2550      
                                                                 
 dense_11 (Dense)            (None, 1)                

# Used reused layers